In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import cv2 as cv
from dataset import CustomImageDataset
from torch.utils.data import DataLoader
from skimage import exposure
from skimage.exposure import match_histograms
import ImageProcessing as IP
import torchvision.transforms as transforms
from pathlib import Path

Declaring image directories

In [2]:
RGB_dir ='./Sample RGB/'
Depth_dir = './Sample Depth/'
ref_dir = './Manhatta Frames/'

Creating custom dataset

In [3]:
def read_images(img_dir, depth_dir):
    """Function that reads images from a given directory"""
    
    img_path = Path(img_dir)
    depth_path = Path(depth_dir)
    
    img_list = []
    depth_list = []

    for img in img_path.iterdir():

        image = cv.imread(img_dir + '/' + img.name, cv.IMREAD_ANYDEPTH + cv.IMREAD_COLOR)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        img_list.append(image)

    for depth in depth_path.iterdir():

        depth = cv.imread(depth_dir + '/' + depth.name, cv.IMREAD_ANYDEPTH + cv.IMREAD_GRAYSCALE)
        depth_list.append(depth)

    return img_list, depth_list

In [4]:
data, labels = read_images(RGB_dir, Depth_dir)

In [5]:
from sklearn.model_selection import train_test_split
train_ims, test_ims, train_labels, test_labels = train_test_split(data, labels, test_size=.2)

In [6]:
#import importlib
#importlib.reload(IP)
#
#data = CustomImageDataset(img_dir= RGB_dir, depth_dir= Depth_dir, transform= IP.rgb_transform, histogram_matching= IP.histogram_matching)
#data_loader = DataLoader(data, batch_size=1, shuffle=True) #Loads a random RGB, Depth couple from the dataset
#rgb_img, depth_img = next(iter(data_loader)) #Unsqueezes and converts to tensor, may need to squeeze(0)
#plt.imshow(rgb_img[0][0,:,:].squeeze().numpy(), cmap='gray')
#plt.show()
#plt.imshow(depth_img[0], cmap='gray')
#plt.show()

Transforming RGB images

In [7]:
#import importlib
#importlib.reload(IP)
#
#test_img = IP.rgb_transform(rgb_img[0])
#plt.imshow(test_img.transpose(0,2).transpose(0,1), cmap='gray')

Matching histogram

In [8]:
#matched_img = IP.histogram_matching(test_img, ref_dir)
#print(np.array(matched_img.shape[::-1]))
#matched_img.transpose_(0,2).transpose_(0,1)
#plt.imshow(matched_img.numpy(), cmap='gray')

Processing a scene for consistent depth prediction

In [9]:
#for i,img in enumerate(rgb_img):
#    cv.imwrite('./consistent_depth-main/color_full/' + str(i) + '.jpeg' , rgb_img[i].squeeze().numpy())
#

### Fine-tuning huggingface model: Intel/dpt-large

In [10]:
from transformers import AutoFeatureExtractor, DPTForDepthEstimation
from PIL import Image

extractor = AutoFeatureExtractor.from_pretrained("Intel/dpt-large")

model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large")

Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
train_dataset = CustomImageDataset(train_ims, train_labels, transform=IP.rgb_transform, histogram_matching=IP.histogram_matching)
test_dataset = CustomImageDataset(test_ims, test_labels, transform=IP.rgb_transform, histogram_matching=IP.histogram_matching)

In [12]:
#from transformers import TrainingArguments, Trainer
#from datasets import load_metric
#
#training_args = TrainingArguments(output_dir="test_trainer")
#metric = load_metric("accuracy")
#
#def compute_metrics(eval_pred):
#    logits, labels = eval_pred
#    predictions = np.argmax(logits, axis=-1)
#    return metric.compute(predictions=predictions, references=labels)
#
#training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
#
#inputs = np.empty(rgb_img.shape)
#for i in range(0,rgb_img.shape[0]):
#    inputs[i] = extractor(images=IP.transform_to_PIL(rgb_img[0]), return_tensors="pt")
#
#trainer = Trainer(
#    model=model,
#    args=training_args,
#    train_dataset= extractor(images=IP.transform_to_PIL(depth_img[0]), return_tensors="pt"),
#    eval_dataset=depth_img[0],
#    #compute_metrics=compute_metrics,
#)
#

In [13]:
#trainer.train()

In [14]:
#from transformers import TrainingArguments, Trainer
#inputs = extractor(images=rgb_img[0], return_tensors="pt")
#
#with torch.no_grad():
#    outputs = model(**inputs)
#    predicted_depth = outputs.predicted_depth
#print(type(predicted_depth))
#
#prediction = torch.nn.functional.interpolate(
#    predicted_depth.unsqueeze(0),
#    size=rgb_img.size,
#    mode="bicubic",
#    align_corners=False,
#)
#
## visualize the prediction
#output = prediction.squeeze().cpu().numpy()
#formatted = (output * 255 / np.max(output)).astype("uint8")
#depth = Image.fromarray(formatted)
#
#depth.show()

In [15]:
#depth_img.shape

In [16]:
from transformers import DPTFeatureExtractor, DPTForDepthEstimation
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import torch
import numpy as np
from PIL import Image
import requests

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="test", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_dataset,
    eval_dataset= test_dataset,
    compute_metrics=compute_metrics,
)


In [17]:
trainer.train()

d:\Anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 168
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 63


  0%|          | 0/63 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 4)